In [1]:
import time
import os
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt

from siriuspy.search import IDSearch
from siriuspy.devices import IVU

import epics

# BEFORE IOC CORRECTION
# epics.caput('SI-08SB:ID-IVU18:UN_Reset.HIGH', 1)
# epics.caput('SI-08SB:ID-IVU18:UN_Stop.HIGH', 1)
# epics.caput('SI-08SB:ID-IVU18:UN_Start.HIGH', 1)

# epics.caput('SI-08SB:ID-IVU18:Reset-Cmd.HIGH', 1)
# epics.caput('SI-08SB:ID-IVU18:Abort-Cmd.HIGH', 1)
# epics.caput('SI-08SB:ID-IVU18:KParamChange-Cmd.HIGH', 1)

In [2]:
def initialize_ivu(beamline):
    # Search ID
    devnameivu = IDSearch.conv_beamline_2_idname(beamline=beamline)
    ivu = IVU(devname=devnameivu)

    # Disable beamline control
    ivu.cmd_beamline_ctrl_disable()
    print('beamline control: ', ivu.is_beamline_ctrl_enabled)

    # Disable center mode and pitch mode
    ivu.set_center_mode(False)
    ivu.set_pitch_mode(False)
    time.sleep(0.5)
    print('center mode: ', ivu.center_mode_status)
    print('pitch mode: ', ivu.pitch_mode_status)

    # Set gap speed
    ivu.set_gap_speed(0.1)
    time.sleep(0.5)
    print('gap speed: {:.3f} mm/s'.format(ivu.gap_speed))

    # Set gap to parked condition
    ivu.set_gap(ivu.kparameter_parked)
    time.sleep(0.5)
    print('gap: {:.3f} mm'.format(ivu.gap))

    return ivu

def move_ivu_gap(ivu:IVU, gap, timeout, verbose=False):
    ivu.set_gap(gap)
    time.sleep(0.5)
    print('Gap-RB {:.3f} mm'.format(ivu.gap)) if verbose else 0
    if ivu.cmd_move_gap_start(timeout):
        time.sleep(0.5)
        print('Undulator is moving...') if verbose else 0
        while ivu.is_moving:
            time.sleep(0.1)
            print('Current gap {:.3f} mm.'.format(ivu.gap_mon), end='\r') if verbose else 0
        print('Gap {:.3f} mm reached.'.format(ivu.gap)) if verbose else 0
        return True
    else:
        print('Error while cmd_move_start.')
        return False

def move_ivu_gap_robust(ivu:IVU, gap, timeout, maxiter=3, verbose=False):
    sucess = move_ivu_gap(ivu, gap=gap, timeout=timeout, verbose=verbose)
    i=0
    while not sucess and i<maxiter:
        i += 1
        print('Trying {:0f}/{:.0f}'.format(i, maxiter)) if verbose else 0
        ivu.cmd_reset(timeout=timeout)
        time.sleep(0.5)
        sucess = move_ivu_gap(ivu, gap=gap, timeout=timeout, verbose=verbose)
    if sucess:
        print('Movimentation done!\n')
        return True
    else:
        print('Error while moving.\n')
        return False


## Search devnames

In [3]:
devnameivu_ema = IDSearch.conv_beamline_2_idname(beamline='EMA')
devnameivu_paineira = IDSearch.conv_beamline_2_idname(beamline='PAINEIRA')

## Initialize EMA IVU

In [4]:
ivu_ema = initialize_ivu(beamline='EMA')

beamline control:  False
center mode:  0
pitch mode:  0
gap speed: 0.100 mm/s
gap: 24.000 mm


In [36]:
timeout = 3
deltat = 180
t0 = time.time()
while datetime.now().hour < 14:
    if deltat >= 180:
        t0 = time.time()
        gap = 5*np.random.random(1) + 20
        speed = 1*np.random.random(1) + 0.01
        ivu_ema.set_gap_speed(speed, timeout)
        time.sleep(0.5)
        sucess = move_ivu_gap_robust(ivu_ema, gap=gap, timeout=timeout, verbose=True)
        if not sucess:
            break
    t = time.time()
    deltat = t - t0
    time.sleep(1)
    print('waiting...', end='\r')
    


Gap-RB 20.142 mm
Undulator is moving...
Gap 20.142 mm reached.
Movimentation done!

Gap-RB 21.115 mm
Undulator is moving...
Gap 21.115 mm reached.
Movimentation done!

Gap-RB 22.662 mm
Undulator is moving...
Gap 22.662 mm reached.
Movimentation done!

Gap-RB 24.000 mm
Undulator is moving...
Gap 24.000 mm reached.
Movimentation done!

Gap-RB 22.832 mm
Undulator is moving...
Gap 22.832 mm reached.
Movimentation done!

Gap-RB 21.871 mm
Undulator is moving...
Gap 21.871 mm reached.
Movimentation done!

Gap-RB 22.254 mm
Undulator is moving...
Gap 22.254 mm reached.
Movimentation done!

Gap-RB 20.160 mm
Undulator is moving...
Gap 20.160 mm reached.
Movimentation done!

Gap-RB 21.766 mm
Undulator is moving...
Gap 21.766 mm reached.
Movimentation done!

Gap-RB 22.418 mm
Undulator is moving...
Gap 22.418 mm reached.
Movimentation done!

Gap-RB 21.240 mm
Undulator is moving...
Gap 21.240 mm reached.
Movimentation done!

Gap-RB 21.644 mm
Undulator is moving...
Gap 21.644 mm reached.
Movimentation

## Initialize PAINEIRA IVU

In [4]:
ivu = IVU(devname=devnameivu_paineira)
ivu.PROPERTIES_DEFAULT

('KParamTaper-Mon',
 'KParam-Mon',
 'MaxVelo-SP',
 'Reset-Cmd',
 'Abort-Cmd',
 'CenterOffset-Mon',
 'BeamLineCtrlEnbl-Sts',
 'CenterOffset-SP',
 'PitchOffset-SP',
 'KParam-RB',
 'KParamVelo-RB',
 'PitchOffset-RB',
 'BeamLineCtrlEnbl-Sel',
 'PitchMode-Sts',
 'KParamAcc-SP',
 'MaxAcc-RB',
 'CenterOffset-RB',
 'PitchMode-Sel',
 'MaxAcc-SP',
 'KParamAcc-RB',
 'KParamTaper-RB',
 'CenterMode-Sts',
 'PitchOffset-Mon',
 'KParamTaper-SP',
 'MaxVelo-RB',
 'Moving-Mon',
 'KParamVelo-SP',
 'CenterMode-Sel',
 'KParamParked-Cte',
 'KParam-SP',
 'KParamChange-Cmd')

In [5]:
print('period: {:.1f} mm'.format(ivu.period_length))
print('polarization mon: '.format(ivu.polarization_mon))
print('conection:', ivu.connected)
ivu.disconnected_pvnames

period: 18.5 mm
polarization mon: 
conection: False


{'SI-14SB:ID-IVU18:KParamAcc-RB',
 'SI-14SB:ID-IVU18:KParamAcc-SP',
 'SI-14SB:ID-IVU18:MaxAcc-RB',
 'SI-14SB:ID-IVU18:MaxAcc-SP',
 'SI-14SB:ID-IVU18:MaxVelo-RB',
 'SI-14SB:ID-IVU18:MaxVelo-SP'}

## Mov tests

In [24]:
ivu = initialize_ivu(beamline='PAINEIRA')

beamline control:  False
center mode:  0
pitch mode:  0
gap speed: 0.100 mm/s
gap: 24.000 mm


In [ ]:
ivu.set_gap_speed(0.5)

In [225]:
move_ivu_gap(ivu, gap=24, timeout=3, verbose=True)
print('here')

Gap-RB 24.000 mm
Undulator is moving...
Gap 24.000 mm reached.
here


In [224]:
move_ivu_gap_robust(ivu, gap=23, timeout=3, maxiter=3, verbose=True)
print('here')

Gap-RB 23.000 mm
Undulator is moving...
Gap 23.000 mm reached.
Movimentation done!
here


In [254]:
gaps0 = np.arange(24, 3, -1)
gaps1 = gaps0[::-1]
gaps = np.concatenate((gaps0, gaps1))
gaps

array([24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10,  9,  8,
        7,  6,  5,  4,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24])

In [255]:

for i, gap in enumerate(gaps):
    sucess = move_ivu_gap_robust(ivu, gap=gap, timeout=timeout, verbose=True)
    if not sucess:
        break


Gap-RB 24.000 mm
Undulator is moving...
Gap 24.000 mm reached.
Movimentation done!

Gap-RB 23.000 mm
Undulator is moving...
Gap 23.000 mm reached.
Movimentation done!

Gap-RB 22.000 mm
Undulator is moving...
Gap 22.000 mm reached.
Movimentation done!

Gap-RB 21.000 mm
Undulator is moving...
Gap 21.000 mm reached.
Movimentation done!

Gap-RB 20.000 mm
Undulator is moving...
Gap 20.000 mm reached.
Movimentation done!

Gap-RB 19.000 mm
Undulator is moving...
Gap 19.000 mm reached.
Movimentation done!

Gap-RB 18.000 mm
Undulator is moving...
Gap 18.000 mm reached.
Movimentation done!

Gap-RB 17.000 mm
Undulator is moving...
Gap 17.000 mm reached.
Movimentation done!

Gap-RB 16.000 mm
Undulator is moving...
Gap 16.000 mm reached.
Movimentation done!

Gap-RB 15.000 mm
Undulator is moving...
Gap 15.000 mm reached.
Movimentation done!

Gap-RB 14.000 mm
Undulator is moving...
Gap 14.000 mm reached.
Movimentation done!

Gap-RB 13.000 mm
Undulator is moving...
Gap 13.000 mm reached.
Movimentation